# Generate data

In [14]:
#from line_profiler import LineProfiler
import torch
from torch.utils.data import Dataset, DataLoader
from scipy.stats import percentileofscore

import numpy as np
import pandas as pd
import plotnine as gg
import torch
import torch.optim as optim
#from dataloader import TabularDataset
from sklearn.ensemble import RandomForestClassifier

import higher

from torch import nn
# from acme.utils.loggers.terminal import TerminalLogger
import dataclasses
#import chex
#import haiku as hk
#import jax
#import jax.numpy as jnp
#import optax
import pandas as pd
#import warnings
import gpytorch
from torch.distributions.bernoulli import Bernoulli
#warnings.filterwarnings('ignore')
#import pipeline_var_l2_loss
import seaborn as sns
#from dataloader import TabularDataset
#from var_l2_loss_estimator import *
#from ENN import basenet_with_learnable_epinet_and_ensemble_prior
import matplotlib.pyplot as plt

#wandb.init()
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import torch.nn.init as init
from datetime import datetime
 
#data from https://drive.google.com/drive/u/1/folders/1WuXIzpYLrLNH6pn9zBMx6oRCCQPRz0F1

directory = '/shared/share_mala/data/eicu_train_test/'
train_csv = 'eicu_train_final.csv'
test_csv = 'eicu_test_final.csv'

df_train = pd.read_csv(directory + train_csv)
df_test = pd.read_csv(directory + test_csv)
df = pd.concat([df_train, df_test], axis = 0)
X_col = list(df.columns)[:-1]
Y = 'EVENT_LABEL'


#df = df.groupby('EVENT_LABEL', group_keys=False).apply(lambda x: x.sample(2000))
Y_data = np.array(df[Y])

'''
For real data -
Select 10 features - use the old file  — code we have - we need to change the code somewhat
Change the setting to a continuous one - use the 
Form the clusters - how many? - 51 clusters
Reduce the dataset size -  use the old file — on fly 

Train points - 100 - 500
Pool points - 1 cluster - 250 points |||||  50 clusters 5 points each 
Number of clusters - 1+50
Batch size to be acquired  - 5
Horizons - 5 horizons

Marginal distribution of x is not same as given dataset
Test points - 1 cluster - 20 points
                     25 clusters - 20 points
                       25 clusters  - 2 points
'''

'\nFor real data -\nSelect 10 features - use the old file  — code we have - we need to change the code somewhat\nChange the setting to a continuous one - use the \nForm the clusters - how many? - 51 clusters\nReduce the dataset size -  use the old file — on fly \n\nTrain points - 100 - 500\nPool points - 1 cluster - 250 points |||||  50 clusters 5 points each \nNumber of clusters - 1+50\nBatch size to be acquired  - 5\nHorizons - 5 horizons\n\nMarginal distribution of x is not same as given dataset\nTest points - 1 cluster - 20 points\n                     25 clusters - 20 points\n                       25 clusters  - 2 points\n'

In [21]:
model = RandomForestClassifier(
n_estimators=100,  # Number of trees in the forest
criterion='gini',  # Function to measure the quality of a split. Can also be 'entropy'.
max_depth=None,    # Maximum depth of the tree. If None, nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
min_samples_split=2,  # Minimum number of samples required to split an internal node
min_samples_leaf=1,   # Minimum number of samples required to be at a leaf node
bootstrap=True,       # Whether bootstrap samples are used when building trees
oob_score=False,      # Whether to use out-of-bag samples to estimate the generalization score
random_state=None,    # Controls both the randomness of the bootstrapping and the sampling of features to consider when looking for the best split
verbose=0,            # Controls the verbosity of the process
class_weight=None,    # Weights associated with classes. Can be 'balanced'.
)
    
def get_feat_importance(model, X_col): # get most important features, input is a randomforest classifier/regressor
    importances = model.feature_importances_
    forest_importances = pd.Series(importances, index = X_col)
    forest_importances = forest_importances.nlargest(10)  #only keep most important 10 feature
    feature_importances = list(forest_importances.index)
    return feature_importances #return the columns   


Y_train = df['EVENT_LABEL']
X_col = list(df.columns)
X_col.remove('EVENT_LABEL')
X_train = df[X_col]


model.fit(X_train, Y_train)
X_col = list(X_train.columns)
X_col_important = get_feat_importance(model, X_col)
# X_col_important.append('EVENT_LABEL')

X_train_imp = df[X_col_important]

Y_pred = model.predict_proba(X_train)
Y_pred = Y_pred[:,1]

In [68]:
# Normalize data
n_cluster = 51
data = np.array(X_train_imp)
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
kmeans = KMeans(n_clusters = n_cluster, random_state = 1)
kmeans.fit(data_normalized)

# y_kmeans = kmeans.predict(data_normalized)

# # Visualize the clusters
# plt.scatter(data[:, 0], data[:, 1], c=y_kmeans, s=50, cmap='viridis')

# # Plot the centroids of the clusters
# centers = kmeans.cluster_centers_
# plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.75)

# plt.xlabel('Feature 1')
# plt.ylabel('Feature 2')
# plt.title('KMeans Clustering')
# plt.show()

/user/ym2865/.conda/envs/yuanzhe_new/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


KMeans(n_clusters=51, random_state=1)

In [70]:
y_kmeans = kmeans.predict(data_normalized)
# Get unique elements and their counts
unique_elements, counts = np.unique(y_kmeans, return_counts=True)

# Zip the unique elements and their counts together for easy iteration
frequency = dict(zip(unique_elements, counts))

# Print frequency of each element
for element, count in frequency.items():
    print(f"Element {element} occurs {count} times.")
    
print(np.sum(np.array(list(frequency.values())) >= 800))

Element 0 occurs 983 times.
Element 1 occurs 3891 times.
Element 2 occurs 840 times.
Element 3 occurs 337 times.
Element 4 occurs 1168 times.
Element 5 occurs 498 times.
Element 6 occurs 3068 times.
Element 7 occurs 1697 times.
Element 8 occurs 1381 times.
Element 9 occurs 523 times.
Element 10 occurs 1921 times.
Element 11 occurs 1421 times.
Element 12 occurs 446 times.
Element 13 occurs 3547 times.
Element 14 occurs 1262 times.
Element 15 occurs 502 times.
Element 16 occurs 388 times.
Element 17 occurs 626 times.
Element 18 occurs 1660 times.
Element 19 occurs 1646 times.
Element 20 occurs 303 times.
Element 21 occurs 7 times.
Element 22 occurs 661 times.
Element 23 occurs 3308 times.
Element 24 occurs 54 times.
Element 25 occurs 668 times.
Element 26 occurs 1255 times.
Element 27 occurs 179 times.
Element 28 occurs 2840 times.
Element 29 occurs 140 times.
Element 30 occurs 86 times.
Element 31 occurs 2981 times.
Element 32 occurs 218 times.
Element 33 occurs 1821 times.
Element 34 o

# Select points from clusters

In [130]:
# < 800 choose 1 test point
# > 800 10 test points
# cluster 1 as training cluster 100 points in training
# 250 points from 1 as pool data
# 5 points from each of the rest for pool

big_cluster_indices = [i for i in range(n_cluster) if frequency[i] >= 800]
small_cluster_indices = [i for i in range(n_cluster) if frequency[i] < 800]
biggest_cluster = 1
big_cluster_indices.remove(biggest_cluster)
n = df.shape[0]
labels = kmeans.labels_
centers = kmeans.cluster_centers_



def close_point_cluster(data, labels, centers, num_closest_points, desired_cluster_index):
    # Index of the desired cluster center
    X = data[labels == desired_cluster_index]

    desired_cluster_index = desired_cluster_index  # Change this to the desired cluster index

    # Calculate distances from each point to the desired cluster center
    distances_to_desired_center = np.linalg.norm(X - centers[desired_cluster_index], axis=1)

    # Sort the distances and select the indices of the 10 closest points
    closest_indices = np.argsort(distances_to_desired_center)[:num_closest_points]


    return closest_indices

def list_remove(arr, index_to_remove, version = 0):
    if version == 0:
        new_arr = np.concatenate((arr[:index_to_remove], arr[index_to_remove+1:]))
    else:
        new_arr = [arr[i] for i in range(len(arr)) if i not in index_to_remove]
    return new_arr

training_pool_indices = close_point_cluster(data_normalized, labels, centers, num_closest_points = 350, desired_cluster_index = biggest_cluster)
shuffled_integers = np.random.permutation(np.arange(0, len(training_pool_indices)))
# Select the first n integers from the shuffled array
n = 100  # Number of random integers to generate
random_integers = shuffled_integers[:n]

train_indices = training_pool_indices[random_integers]
pool_indices = list_remove(training_pool_indices, random_integers, 1)


test_pool_indices_small = [close_point_cluster(data_normalized, labels, centers, num_closest_points = 6, desired_cluster_index = i)  for i in small_cluster_indices]
random_indices_list = np.random.randint(low=0, high=6, size=n_cluster)
pool_indices_small = [test_pool_indices_small[i][random_indices_list[i]] for i in range(len(small_cluster_indices))]
test_indices_small = [list_remove(test_pool_indices_small[i],random_indices_list[i]) for i in range(len(small_cluster_indices))]

test_pool_indices_big = [close_point_cluster(data_normalized, labels, centers, num_closest_points = 15, desired_cluster_index = i)  for i in big_cluster_indices]
#random_indices_list_big = np.random.randint(low=0, high=15, size=(len(big_cluster_indices),5))
random_indices_list_big = []
for i in range(len(big_cluster_indices)):
    shuffled_integers = np.random.permutation(np.arange(0, 15))
    # Select the first n integers from the shuffled array
    n = 5  # Number of random integers to generate
    random_integers = shuffled_integers[:n]
    random_indices_list_big.append(random_integers)


pool_indices_big = [test_pool_indices_big[i][random_indices_list_big[i]] for i in range(len(big_cluster_indices))]
test_indices_big = [list_remove(test_pool_indices_big[i],random_indices_list_big[i], version = 1) for i in range(len(big_cluster_indices))]

 
#train_indices
pool_indices_all = np.concatenate((pool_indices, pool_indices_small, np.array(pool_indices_big).flatten()))
test_indices_all = np.concatenate((np.array(test_indices).flatten(), np.array(test_indices_small).flatten(), np.array(test_indices_big).flatten()))


In [138]:
data_dir = '/shared/share_mala/data/eicu_train_test/0518_data/'

index_list = [train_indices, pool_indices_all, test_indices_all]
name_list = ['train','pool','test']

for i in range(3):
    pd.DataFrame(data_normalized[index_list[i]]).to_csv(data_dir + name_list[i] + '_x.csv')
    pd.DataFrame(Y_pred[index_list[i]]).to_csv(data_dir + name_list[i] + '_y.csv')